# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.  The S&P 500 is market capital-weighted, so larger companies are over-represented and smaller companies are under-represented.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [3]:
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


Some stocks have [dual stock structures](https://www.investopedia.com/terms/d/dualclassstock.asp) (for example, Google has Class-A and Class-B shares), which is why there are more than 500 stocks.

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [9]:
symbol="AAPL"
api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url)
print(data.status_code)
data = data.json()
print(data)

200
{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'S NLD/ OENTAS)GATAGEELRKAL MSQBC(', 'calculationPrice': 'close', 'open': 137, 'openTime': 1620560346623, 'openSource': 'fficaiol', 'close': 135.51, 'closeTime': 1675883659193, 'closeSource': 'ilciafof', 'high': 137.5, 'highTime': 1680121486467, 'highSource': 'neutce5 ildierd1pme a y', 'low': 134.73, 'lowTime': 1664021296400, 'lowSource': 'ecideart5yeliemn1 d  up', 'latestPrice': 138.03, 'latestSource': 'Close', 'latestTime': 'December 31, 2020', 'latestUpdate': 1687097669110, 'latestVolume': 102514731, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 135.91, 'delayedPriceTime': 1678695159196, 'oddLotDelayedPrice': 136.28, 'oddLotDelayedPriceTime': 1649608932698, 'extendedPrice': 138.68, 'extendedChange': 0.05, 'extendedChangePercent': 0.00039, 'extendedPriceTime': 1670273931812, 'previousClose': 138.54, 'previousVolume': 96871609, 'change': -1.04, 'changePercent': -0.008, 'v

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [12]:
price = data['latestPrice']
print(price)
market_cap = data['marketCap']
print(market_cap)

138.03
2275592911654


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [42]:
# function to get dict with info about each stock
def get_stock(symbol):
    api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
    return requests.get(api_url).json()


In [43]:
# test get_stock function
for symbol in stocks['Ticker'][0:5]:
    data = get_stock(symbol)
    print(f"{symbol}: price {data['latestPrice']}, market_cap {data['marketCap']} ")
    

A: price 123.83, market_cap 37381151477 
AAL: price 16.32, market_cap 9901269709 
AAP: price 159.08, market_cap 11163593077 
AAPL: price 135.18, market_cap 2364776393111 
ABBV: price 107.88, market_cap 193726349158 


In [44]:
# create dataframe
my_columns = ('ticker', 'stock_price', 'market_capitalization', 'n_shares')
final_df = pd.DataFrame(columns = my_columns)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [45]:
# only do the first five because this is really slow
for symbol in stocks['Ticker'][0:5]:
    data = get_stock(symbol)
    final_df = final_df.append(
        pd.Series(
            [symbol, data['latestPrice'], data['marketCap'], 'N/A'],
            index=my_columns
        ),
        ignore_index=True
    )

In [46]:
print(final_df)

  ticker  stock_price market_capitalization n_shares
0      A       123.48           36791280522      N/A
1    AAL        16.51            9943410231      N/A
2    AAP       160.04           11108262226      N/A
3   AAPL       135.43         2294823103861      N/A
4   ABBV       109.86          198065501911      N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [37]:
#https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [54]:
# https://iexcloud.io/docs/api/#batch-requests
def get_batch(symbol_list):
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={','.join(symbol_list)}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(url).json()



In [59]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
print(symbol_groups[1])
#print(get_batch(symbol_groups[0]))
print(get_batch(symbol_groups[0])['A'])

100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object
{'quote': {'symbol': 'A', 'companyName': 'Agilent Technologies Inc.', 'primaryExchange': 'OER N WSOHNKIE CCEX, .GTYACKN', 'calculationPrice': 'close', 'open': 119.62, 'openTime': 1627951319763, 'openSource': 'ioiflfca', 'close': 120.6, 'closeTime': 1650647410432, 'closeSource': 'folicafi', 'high': 120.2, 'highTime': 1665209789460, 'highSource': 're  euitdmedyencpl 51ia', 'low': 120.34, 'lowTime': 1653433255745, 'lowSource': '5rndyticed ea ee 1miupl', 'latestPrice': 123.04, 'latestSource': 'Close', 'latestTime': 'December 31, 2020', 'latestUpdate': 1640367170555, 'latestVolume': 1353932, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 119.39, 'delayedPriceTime': 1635595107976, 'oddLotDelayedPrice': 123.08, 'oddLotDelayedPriceTime': 1625079721888, 'extendedPrice': 120.18, 'ex

In [60]:
# create dataframe
my_columns = ('ticker', 'stock_price', 'market_capitalization', 'n_shares')
final_df = pd.DataFrame(columns = my_columns)

for symbol_list in chunks(stocks['Ticker'], 100):
    data = get_batch(symbol_list)
    for symbol in data:
        final_df = final_df.append(
        pd.Series(
            [symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'N/A'],
            index=my_columns
        ),
        ignore_index=True
    ) 

In [61]:
print(final_df)

    ticker  stock_price market_capitalization n_shares
0        A       123.69           37523758197      N/A
1      AAL        16.01            9881590306      N/A
2      AAP       164.40           10916457531      N/A
3     AAPL       139.19         2285425949314      N/A
4     ABBV       110.54          192663522798      N/A
..     ...          ...                   ...      ...
500    YUM       111.11           34027277521      N/A
501    ZBH       158.07           33119801615      N/A
502   ZBRA       393.49           21229515412      N/A
503   ZION        44.65            7151705916      N/A
504    ZTS       173.70           82166275075      N/A

[505 rows x 4 columns]


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [77]:
# get total dollar value of portfolio
portfolio_size = ''

while not isinstance(portfolio_size, float):
    try:
        portfolio_size = float(input('Enter the size of your portfolio:'))
    except ValueError:
        print("please enter an integer")

Enter the size of your portfolio:1000000


In [82]:
# number of shares in each stock to buy is total number of shares divided by portfolio size
position_size = portfolio_size / len(final_df)

In [83]:
# loop through dataframe
for i in range(len(final_df.index)):
    price = final_df.loc[i, 'stock_price']
    final_df.loc[i, 'n_shares'] = math.floor( position_size / price)


In [80]:
final_df

,ticker,stock_price,market_capitalization,n_shares
0,A,123.69,37523758197,16
1,AAL,16.01,9881590306,123
2,AAP,164.40,10916457531,12
3,AAPL,139.19,2285425949314,14
4,ABBV,110.54,192663522798,17
...,...,...,...,...
500,YUM,111.11,34027277521,17
501,ZBH,158.07,33119801615,12
502,ZBRA,393.49,21229515412,5
503,ZION,44.65,7151705916,44


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [115]:
writer = pd.ExcelWriter('Recommended_trades.xlsx', engine='xlsxwriter')

In [116]:
final_df.to_excel(writer, 'Recommended trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [117]:
bg_colour = '#0a0a23'
font_colour = '#ffffff'

string_format = writer.book.add_format({
    'font_color': font_colour,
    'bg_color': bg_colour,
    'border': 1,
})

dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_colour,
    'bg_color': bg_colour,
    'border': 1,
})

integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_colour,
    'bg_color': bg_colour,
    'border': 1,
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [108]:
#writer.sheets['Recommended trades'].set_column('A:A', 18, string_format)
#writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [118]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', integer_format],
    'D': ['Number of shares', integer_format]
}

In [119]:
for column in column_formats:
    writer.sheets['Recommended trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended trades'].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [120]:
writer.save()